1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [152]:
def hex_letters(n): # or, with dictionaries and get(*args)
    assert type(n) == int
    if n == 10: n = 'A'
    elif n == 11: n = 'B'
    elif n == 12: n = 'C'
    elif n == 13: n = 'D'
    elif n == 14: n = 'E'
    elif n == 15: n = 'F'
    else: n = str(n)
    return n

def hex_digits(l):
    if l == 'A': l = 10
    elif l == 'B': l = 11
    elif l == 'C': l = 12
    elif l == 'D': l = 13
    elif l == 'E': l = 14
    elif l == 'F': l = 15
    else: l = int(l)
    return l

print(hex_letters(0))
print(hex_digits(9))

0
9


In [147]:
def approx(n,m):
    if m>=5:
        n+=1
    else:
        n=n
    return n

In [1]:
def converter(x, inrepr, outrepr):
   # assert (type(x) == int or type(x) == float) and (x>=0), "x must be non-negative int or float" # all possible options except complex numbers
    floor_x_str = ''
    fractional_x_str = ''
     # leads to unstable results on decimal digits from bin to dec - fix ?
    
        
    if inrepr == "dec" and outrepr == "bin": 
        floor_x = int(x // 1)
        fractional_x = x - x // 1
        while floor_x > 0:
            remainder = floor_x%2
            floor_x = floor_x//2
            floor_x_str += str(remainder)
        floor_x_str = floor_x_str[::-1]
        for i in range(5): # fixed number of fractionary digits
            fractional_x *= 2
            remainder = int(fractional_x // 1)
            fractional_x = fractional_x - fractional_x // 1 
            fractional_x_str += str(remainder)
        result = float(floor_x_str + '.' + fractional_x_str)
        
    elif inrepr == "dec" and outrepr == "hex":
        floor_x = int(x // 1)
        fractional_x = x - x // 1
        if floor_x == 0: floor_x_str = '0'
        else:
            while floor_x > 0:
                remainder = hex_letters(floor_x%16)
                floor_x = floor_x//16
                floor_x_str += remainder
            floor_x_str = floor_x_str[::-1]
        for i in range(5): # fixed number of significant digits
            fractional_x *= 16
            remainder = hex_letters(int(fractional_x // 1))
            fractional_x = fractional_x - fractional_x // 1 
            fractional_x_str += remainder
        result = floor_x_str + '.' + fractional_x_str
        
    elif inrepr == "bin" and outrepr == "dec":
        floor_x = int(x // 1)
        fractional_x = x - x // 1
        floor_result = 0
        for i,digit in enumerate(str(floor_x)[::-1]):
            floor_result += int(digit) * 2**i
        fractional_result = 0
        fractional_x_str = str(fractional_x)+'000000'
        for i in range(2,7):
            digit = str(fractional_x_str[i])
            #assert (digit == '1' or digit == '0'), "Input must be binary float or int"
            fractional_result += int(digit) * 2**(-i+1)
        result = floor_result + fractional_result

    elif inrepr == "hex" and outrepr == "dec":
        split_x = str(x).split('.')
        floor_x = [d for d in split_x[0]]
        fractional_x = [d for d in split_x[1]] + [0,0,0,0,0,0]
        floor_result = 0
        for i,digit in enumerate(floor_x[::-1]):
            floor_result += int(hex_digits(digit)) * 16**i
        fractional_result = 0
        for i in range(0,5):
            digit = str(fractional_x[i])
            #assert (digit == '1' or digit == '0'), "Input must be binary float or int"
            fractional_result += hex_digits(digit) * 16**(-i-1)
        result = floor_result + fractional_result
            
    return result
    

print(converter(0, 'dec', 'bin'))
    

0.0


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
# idea: input as string, split it using slice syntax for lists,
# assign to each int value (for sign value it's +-1);
# then result is product of the three, approximate it to one digit using the "approx" function

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
# idea: for loop with counter variable = i, within a try-except block where except prints counter variable (if it can?)

def under_over():
    u=1
    v=1
    count = 0
    for i 

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [ ]:
# idea: while cycle, exceptions not necessary

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
# idea: 
# b - do calculation on paper;
# intuitively not anymore dividing by a small number but by smth proportional to b, at least for the negative solution; also b simplifies at numerator
# c- see if c also handles well "regular" cases;
# also try cases with very big / small b, c; might help to handle diff cases with diff formulas

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
# idea: (straightforward implementation;)
# reason why: dividing by number close to zero accentuates tiny errors in numerator (evm prove it with error propagation)

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
# idea: straightforward and similar to previous two; then just a matter of playing around